In [1]:
from NN import *

### Defining the network

In [2]:
numbers = [str(i) for i in range(2000)]

neurons_per_layer = 10000
neurons_in_attractor = 30
fan_out = 3000
additional_samples = 300

samples = PrimeAttractors(additional_samples, neurons_per_layer, neurons_in_attractor, numbers)

self_weights = ConnectionWeights(neurons_per_layer, neurons_per_layer, fan_out)
one_shot_learned_weights = ConnectionWeights(neurons_per_layer, neurons_per_layer, fan_out)
#one_shot_learned_weights_default = ConnectionWeights(neurons_per_layer, neurons_per_layer, fan_out)

table_layer = Layer(neurons_per_layer)
key_layer = Layer(neurons_per_layer)
hash_layer = Layer(neurons_per_layer)
value_layer = Layer(neurons_per_layer)
#default_layer = Layer(neurons_per_layer)

self_table = Connection(self_weights, table_layer, table_layer, 1.5)
self_key = Connection(self_weights, key_layer, key_layer, 1.5)
self_value = Connection(self_weights, value_layer, value_layer, 1.5)
#self_default = Connection(self_weights, default_layer, default_layer, 1.1)

hashing_connection = SecondOrderConnection(table_layer, neurons_in_attractor, key_layer, neurons_in_attractor, hash_layer, neurons_in_attractor)
table_connection = Connection(one_shot_learned_weights, hash_layer, value_layer, 0.2 * neurons_per_layer / (neurons_in_attractor * fan_out))
#default_to_value = Connection(one_shot_learned_weights_default, default_layer, value_layer, 0.05 * neurons_per_layer / (neurons_in_attractor * fan_out))
#network = Network([table_layer, key_layer, hash_layer, value_layer, default_layer], [self_table, self_key, self_value, self_default, hashing_connection, table_connection, default_to_value])
network = Network([table_layer, key_layer, hash_layer, value_layer], [self_table, self_key, self_value, hashing_connection, table_connection])


### Learning default

In [3]:
#samples.init_states(default_layer, "0")
#samples.init_states(value_layer, "0")
#default_to_value.bind()

### Learning prime attractor weights

In [4]:
def output(cost):
    print(str(100.0 * cost), flush=True)
    return 100.0 * cost < 0.2

costs = self_weights.train(samples, samples, 0.2, output, min_value=-0.2)

for i in range(20):
    e = i / 20
    if np.sum(100.0*costs > e) <= additional_samples:
        samples.samples = samples.samples[100.0*costs <= e, :]
        break
        
#np.sum(costs[100.0*costs <= e]) / np.sum(100.0*costs <= e)

1429.1691598385166
1160.1384103084142
942.2416333873545
765.7834551412324
622.9166329197274
507.29342283869414
413.77427061086297
338.14869095286554
276.9090509149325
227.1509945840373
186.51274380517725
153.12189038837562
125.54794424890581
102.74298610703572
83.92797320234152
68.46630883611499
55.80157199294056
45.44837073711478
36.994323760976705
30.096359491078033
24.472117256665683
19.88988825886309
16.159456509132166
13.12460140973667
10.657122385711654
8.651948694824933
7.023115234133296
5.700399195044006
4.626533177523781
3.754862398131833
3.0474151105126133
2.4733109983472947
2.0074510070990126
1.6294434498538384
1.322730186059986
1.0738688758516604
0.8719456858742684
0.7081059691631271
0.5751639460038399
0.46728937658482683
0.37975311390260713
0.30871845372212436
0.2510732543435755
0.20429422397438787
0.1663346562755576


In [5]:
costs = self_weights.train(samples, samples, 0.0, output, min_value=-0.2)

0.14306386412679212


In [6]:
np.sum(100*costs > 10/20)

0

In [7]:
costs.shape

(2231,)

### Binding and recall functions

In [5]:
def bind(table: str, key: str, value: str):
    table_connection.opened = False
    samples.init_states(table_layer, table)
    samples.init_states(key_layer, key)
    samples.init_states(value_layer, value)
    hash_layer.clear_states()
    for _ in range(2):
        network.tick()
    table_connection.bind()

def recall(table: str, key: str):
    table_connection.opened = True
    samples.init_states(table_layer, table)
    samples.init_states(key_layer, key)
    value_layer.clear_states()
    hash_layer.clear_states()
    for _ in range(20):
        network.tick()

def unbind(table: str, key: str):
    recall(table, key)
    table_connection.unbind()


### Testing

In [6]:
def test(table, key):   
    recall(table, key)
    best, best_score, second, second_score = samples.best_named_attractor(value_layer)
    
    print("best={0} ({1}), second={2} ({3})".format(best, best_score, second, second_score))
    
    return best

def debug(table, key):
    table_connection.opened = True
    samples.init_states(table_layer, table)
    samples.init_states(key_layer, key)
    value_layer.clear_states()
    hash_layer.clear_states()
    for i in range(50):
        network.tick()
        best, best_score, second, second_score = samples.best_named_attractor(value_layer)   
        print("{4}) best={0} ({1}), second={2} ({3})".format(best, best_score, second, second_score, i))

    

In [7]:
bind("1", "2", "3")
test("1", "2")
unbind("1", "2")

best=3 (0.9999999999999999), second=100 (0.06666666666666667)


In [8]:
bind("4", "5", "3")
test("1", "2")
test("4", "5")
unbind("4", "5")

best=0 (0.0), second=0 (0.0)
best=3 (0.9999999999999999), second=100 (0.06666666666666667)


In [9]:
bind("6", "7", "8")
test("1", "2")
test("4", "5")
test("6", "7")
test("0", "1")
unbind("6", "7")

best=0 (0.0), second=0 (0.0)
best=0 (0.0), second=0 (0.0)
best=8 (0.9999999999999999), second=719 (0.06666666666666667)
best=0 (0.0), second=0 (0.0)


In [10]:
for i in range(1, 100):
    print(str(i))
    for j in range(1, 100):
        #unbind(str(i), str(j))
        bind(str(i), str(j), str((i*j)%2000))
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
test("3", "4")

best=12 (0.9999999999999999), second=203 (0.06666666666666667)


'12'

In [12]:
test("1", "2")

best=2 (0.9999999999999999), second=105 (0.06666666666666667)


'2'

In [13]:
test("6", "9")

best=54 (0.9999999999999999), second=218 (0.06666666666666667)


'54'

In [14]:
test("6", "7")

best=42 (0.9999999999999999), second=183 (0.06666666666666667)


'42'

In [15]:
for i in range(29, 100):
    for j in range(1, 100):
        print(str(i*100 + j))
        r = int(test(str(i), str(j)))
        if r != (i * j) % 2000:
            print("****Error****")
            debug(str(i), str(j))
        

2901
best=29 (0.9999999999999999), second=267 (0.06666666666666667)
2902
best=58 (0.9999999999999999), second=332 (0.06666666666666667)
2903
best=87 (0.9999999999999999), second=63 (0.06666666666666667)
2904
best=116 (0.9999999999999999), second=102 (0.06666666666666667)
2905
best=145 (0.9999999999999999), second=1153 (0.06666666666666667)
2906
best=174 (0.9999999999999999), second=560 (0.06666666666666667)
2907
best=203 (0.9999999999999999), second=12 (0.06666666666666667)
2908
best=232 (0.9999999999999999), second=63 (0.06666666666666667)
2909
best=261 (0.9999999999999999), second=365 (0.06666666666666667)
2910
best=290 (0.9999999999999999), second=17 (0.1)
2911
best=319 (0.9999999999999999), second=342 (0.06666666666666667)
2912
best=348 (0.9999999999999999), second=430 (0.06666666666666667)
2913
best=377 (0.9999999999999999), second=462 (0.06666666666666667)
2914
best=406 (0.9999999999999999), second=344 (0.06666666666666667)
2915
best=435 (0.9999999999999999), second=17 (0.0666666

In [ ]:
debug("29", "93")